In [1]:
import pandas as pd

# Creamos las funciones para FastAPI

## Abrimos tablas procesadas anteriormente

- merged_steam_user_data.parquet

In [2]:
# Direccion del archivo comprimido y reconvertido
steam_user_content = 'data/FastAPI/merged_steam_user_data.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
steam_user_data = pd.read_parquet(steam_user_content)

- merged_steam_rev_data.parquet

In [3]:
# Direccion del archivo comprimido y reconvertido
steam_rev_content = 'data/FastAPI/merged_steam_rev_data.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
steam_rev_data = pd.read_parquet(steam_rev_content)

### Verificaciones Iniciales

- steam_user_data

In [4]:
steam_user_data.head(10)

,item_id,user_id,playtime_forever,genres,release_year
0,20570,76561198025089200,3,Strategy,2013
1,12110,napoleoninrags,0,Action Adventure Casual Indie Simulation,2017
2,620,76561198064201971,51,Strategy RPG,2011
3,4000,theicyhawt,460,Adventure,2016
4,20,Kusaryuu,0,Action Indie RPG,2016
5,6000,76561198041874601,0,Casual Indie Simulation Strategy,2016
6,18500,76561198000149660,9,Action,2015
7,2630,76561197964317648,1315,Action Adventure,2015
8,340,76561198025256638,0,Action RPG,2010
9,420,GermanPenguin,84,Simulation,2010


In [5]:
steam_user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3187 entries, 0 to 3186
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           3187 non-null   int64 
 1   user_id           3187 non-null   object
 2   playtime_forever  3187 non-null   int64 
 3   genres            3187 non-null   object
 4   release_year      3187 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 124.6+ KB


In [6]:
valores_unicos = steam_user_data['release_year'].unique()
print(valores_unicos)

[2013 2017 2011 2016 2015 2010 2009 1999 2014 2012 1997 2005 2018 1998
 1995 2001 2008 1988 2003 2006 2007 2002 1994 1996 1986]


- steam_rev_data

In [7]:
steam_rev_data.head(1)

,app_name,item_id,release_year,recommend,sentiment_analysis
0,Sword of the Samurai,327950,1989,True,2


In [8]:
steam_rev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6288 entries, 0 to 6287
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   app_name            6288 non-null   object
 1   item_id             6288 non-null   int64 
 2   release_year        6288 non-null   int64 
 3   recommend           6288 non-null   bool  
 4   sentiment_analysis  6288 non-null   int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 202.8+ KB


## UserForGenre

In [9]:
def UserForGenre(genero: str = 'Action') -> dict:
    df_filtered_by_genre = steam_user_data[steam_user_data['genres']==genero]

    user_hours = df_filtered_by_genre.groupby('user_id')['playtime_forever'].sum()
    max_time_played_user = user_hours.idxmax()

    max_user_df = df_filtered_by_genre[df_filtered_by_genre['user_id']== max_time_played_user]
    hours_by_year = max_user_df.groupby('release_year')['playtime_forever'].sum().reset_index()
    hours_years_list = [{'Año': year, 'Horas': hours} for year, hours in zip(hours_by_year['release_year'], hours_by_year['playtime_forever'])]

    result ={
        'Usuario con mas horas jugadas para genero {}'.format(genero): max_time_played_user,
        'Horas jugadas': hours_years_list
        }
    

    return result

resultado=UserForGenre("Casual")
print(resultado)
print(UserForGenre('Action'))
print(UserForGenre('Adventure'))

{'Usuario con mas horas jugadas para genero Casual': '76561198035993855', 'Horas jugadas': [{'Año': 2015, 'Horas': 1619}]}
{'Usuario con mas horas jugadas para genero Action': 'Shad0wlord', 'Horas jugadas': [{'Año': 2011, 'Horas': 134683}]}
{'Usuario con mas horas jugadas para genero Adventure': '76561198115281134', 'Horas jugadas': [{'Año': 2016, 'Horas': 138596}]}


## PlayTimeGenre

In [10]:
def PlayTimeGenre(genero: str = 'Action') -> dict:
    df_filtered_by_genre = steam_user_data[steam_user_data['genres']==genero]
    user_hours = df_filtered_by_genre.groupby('release_year')['playtime_forever'].sum()
    max_time_played_user = user_hours.idxmax()

    result ={
        "Año de lanzamiento con más horas jugadas para género {}".format(genero): max_time_played_user
        }
    return result

resultado=PlayTimeGenre("Casual")
print(resultado)
print(PlayTimeGenre('Indie'))
print(PlayTimeGenre('Adventure'))

{'Año de lanzamiento con más horas jugadas para género Casual': 2015}
{'Año de lanzamiento con más horas jugadas para género Indie': 2011}
{'Año de lanzamiento con más horas jugadas para género Adventure': 2016}


## UsersRecommend

In [11]:
def UsersRecommend(year: int = '2020') -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year']==year]
    año_recommend=user_reviews_recommend.groupby('item_id')['recommend'].sum()
    usuario_recomienda = año_recommend.idxmax()


    result ={
        "1º Puesto para año {} es para el item".format(year): usuario_recomienda
        }
    return result

resultado=UsersRecommend(2015)
print(resultado)

{'1º Puesto para año 2015 es para el item': 208650}


In [12]:
def UsersRecommend(year: int = 2017) -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year']==year]
    año_recommend=user_reviews_recommend.groupby('app_name')['recommend'].sum()
    usuario_recomienda = año_recommend.idxmax()


    result ={
        "1º Puesto para año {} es para el item".format(year): usuario_recomienda
        }
    return result

resultado=UsersRecommend(2015)
print(resultado)

{'1º Puesto para año 2015 es para el item': '12 Labours of Hercules'}


In [13]:
def Top3RecommendedGames(year: int = 2017) -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    top_games = user_reviews_recommend.groupby('app_name')['recommend'].sum().nlargest(3)
    top_games_list = top_games.index.tolist()

    result = {
        f"Los tres juegos más recomendados para el año {year} son": top_games_list
    }
    return result

resultado = Top3RecommendedGames(2015)
print(resultado)


{'Los tres juegos más recomendados para el año 2015 son': ['12 Labours of Hercules', '404Sight', '8BitMMO']}


In [14]:
def Top3RecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos más recomendados
    top_games = user_reviews_recommend.groupby('app_name')['recommend'].sum().nlargest(3)
    top_games_list = top_games.index.tolist()
    
    # Enumerar los juegos más jugados
    top_played_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(top_games_list)]

    return top_played_ranking

resultado = Top3RecommendedGames(2015)
print(resultado)


[{'Puesto 1: ': '12 Labours of Hercules'}, {'Puesto 2: ': '404Sight'}, {'Puesto 3: ': '8BitMMO'}]


In [15]:
def Top3RecommendedGames(year: int = 2020) -> list:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos más recomendados
    top_games = user_reviews_recommend.groupby('app_name')['recommend'].sum().nlargest(3)
    top_games_list = top_games.index.tolist()
    
    # Enumerar los juegos más jugados
    top_played_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(top_games_list)]

    return top_played_ranking

resultado = Top3RecommendedGames(2015)

print("Mas recomendados:")
for game_dict in resultado:
    for key, value in game_dict.items():
        print(key, value)

Mas recomendados:
Puesto 1:  12 Labours of Hercules
Puesto 2:  404Sight
Puesto 3:  8BitMMO


In [16]:
def Top3LeastRecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos menos recomendados usando nsmallest
    least_recommended = user_reviews_recommend.groupby('item_id')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    # Enumerar los juegos menos recomendados
    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking

resultado = Top3LeastRecommendedGames(2014)
print(resultado)


[{'Puesto 1: ': 222420}, {'Puesto 2: ': 235210}, {'Puesto 3: ': 237970}]


In [17]:
def Top3LeastRecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos menos recomendados usando nsmallest
    least_recommended = user_reviews_recommend.groupby('app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    # Enumerar los juegos menos recomendados
    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking

resultado = Top3LeastRecommendedGames(2014)
print(resultado)

[{'Puesto 1: ': '99 Levels To Hell'}, {'Puesto 2: ': 'A.R.E.S. Extinction Agenda EX'}, {'Puesto 3: ': 'Abyss Odyssey'}]


In [18]:
def Top3LeastRecommendedGames(year: int = 2020) -> dict:
    # Filtrar por el año deseado en user_reviews_dataset
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Obtener los juegos menos recomendados usando nsmallest
    least_recommended = user_reviews_recommend.groupby('app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    # Enumerar los juegos menos recomendados
    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking

resultado = Top3LeastRecommendedGames(2014)
print("Mas recomendados:")
for game_dict in resultado:
    for key, value in game_dict.items():
        print(key, value)

Mas recomendados:
Puesto 1:  99 Levels To Hell
Puesto 2:  A.R.E.S. Extinction Agenda EX
Puesto 3:  Abyss Odyssey


In [19]:
def UsersNotRecommend(year: int = 2017) -> dict:
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    least_recommended = user_reviews_recommend.groupby('app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()

    least_recommended_ranking = [{"Puesto {}: ".format(i + 1): str(game)} for i, game in enumerate(least_recommended_list)]

    return least_recommended_ranking
resultado = UsersNotRecommend(2014)
print(resultado)

[{'Puesto 1: ': '99 Levels To Hell'}, {'Puesto 2: ': 'A.R.E.S. Extinction Agenda EX'}, {'Puesto 3: ': 'Abyss Odyssey'}]


In [20]:
def UsersNotRecommend(year: int = 2017):
    user_reviews_recommend = steam_rev_data[steam_rev_data['release_year'] == year]
    least_recommended = user_reviews_recommend.groupby(
        'app_name')['recommend'].sum().nsmallest(3)
    least_recommended_list = least_recommended.index.tolist()
    least_recommended_ranking = {
        f"Puesto {i + 1}": game for i, game in enumerate(least_recommended_list)}

    return least_recommended_ranking


In [21]:
def sentiment_analysis(year: int = 2020) -> dict:
    # Filtrar por release_year
    year=int(year)
    reviews_by_year = steam_rev_data[steam_rev_data['release_year'] == year]
    
    # Contar las ocurrencias de 0, 1 y 2 en sentiment_analysis
    counts_by_sentiment = reviews_by_year['sentiment_analysis'].value_counts().fillna(0).astype(int)
    
    # Crear el diccionario con los recuentos solicitados
    result = {
        'Negativo': counts_by_sentiment.get(0, 0),
        'Neutro': counts_by_sentiment.get(1, 0),
        'Positivo': counts_by_sentiment.get(2, 0)
    }

    return result

resultado = sentiment_analysis(2017)
print(resultado)

{'Negativo': 34, 'Neutro': 29, 'Positivo': 55}


In [22]:
def sentiment_analysis(year: int = 2017) -> dict:
    reviews_by_year = steam_rev_data[steam_rev_data['release_year'] == year]
    counts_by_sentiment = reviews_by_year['sentiment_analysis'].value_counts(
    ).to_dict()
    result = {
        'Negativo': counts_by_sentiment.get(0, 0),
        'Neutro': counts_by_sentiment.get(1, 0),
        'Positivo': counts_by_sentiment.get(2, 0)
    }

    return result
resultado = sentiment_analysis(2017)
print(resultado)

{'Negativo': 34, 'Neutro': 29, 'Positivo': 55}
